<a href="https://colab.research.google.com/github/AsmaaYassinDev/Behavioural-Anomaly-Detection-for-ATO-Fraud/blob/main/ATO_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
import warnings

# تجاهل التحذيرات غير الهامة
warnings.filterwarnings('ignore', category=UserWarning)

print("--- بدء التنفيذ (الكود المستقر) ---")
print("الهدف: تحديد الحسابات الشاذة (Anomalous Accounts) بناءً على سلوكها.")

# --- تحميل البيانات (تأكدي أن هذا الاسم مطابق لاسم الملف الذي رفعتِه) ---
file_path = 'PS_20174392719_1491204439457_log24.csv'
try:
    df = pd.read_csv(file_path)
    print(f"تم تحميل الملف الكامل بنجاح ({len(df)} صف).")
except Exception as e:
    print(f"حدث خطأ أثناء تحميل البيانات: {e}")
    print("!!! تأكدي من أن اسم الملف في الكود (file_path) هو نفس اسم الملف الذي قمتِ برفعه بالضبط !!!")
    exit()

# --- الخطوة 1: إنشاء "العينة الذكية" (Smart Sample) ---
print("\n--- الخطوة 1: إنشاء 'العينة الذكية' ---")
df_fraud = df[df['isFraud'] == 1]
fraud_dest_ids = df_fraud['nameDest'].unique()
fraud_orig_ids = df_fraud['nameOrig'].unique()
all_fraud_user_ids = np.union1d(fraud_dest_ids, fraud_orig_ids)
df_fraud_lifecycle = df[
    df['nameOrig'].isin(all_fraud_user_ids) |
    df['nameDest'].isin(all_fraud_user_ids)
]

df_normal = df[df['isFraud'] == 0]
sample_size = min(500000, len(df_normal))
df_normal_sample = df_normal.sample(n=sample_size, random_state=42)

df_smart_sample = pd.concat([df_fraud_lifecycle, df_normal_sample]).drop_duplicates(keep='first')
print(f"تم إنشاء 'العينة الذكية' النهائية بحجم {len(df_smart_sample)} صف.")

# --- الخطوة 2: بناء البروفايلات السلوكية (الميزات القوية) ---
print("\n--- الخطوة 2: بناء البروفايلات السلوكية ---")

# (أ) حساب إجمالي المستلم، إجمالي المسحوب، وعدد المرسلين الفريدين
df_received = df_smart_sample[df_smart_sample['type'].isin(['TRANSFER', 'CASH_IN'])]
total_received = df_received.groupby('nameDest')['amount'].sum().to_dict()
unique_senders = df_received.groupby('nameDest')['nameOrig'].nunique().to_dict()

df_cashed_out = df_smart_sample[df_smart_sample['type'] == 'CASH_OUT']
total_cashed_out = df_cashed_out.groupby('nameOrig')['amount'].sum().to_dict()

all_user_ids = set(total_received.keys()) | set(total_cashed_out.keys()) | set(unique_senders.keys())
profiles_list = []
for user_id in all_user_ids:
    received = total_received.get(user_id, 0)
    cashed_out = total_cashed_out.get(user_id, 0)
    senders = unique_senders.get(user_id, 0)

    ratio = (cashed_out / (received + 1e-6))
    ratio = min(ratio, 1.0) # النسبة لا يمكن أن تزيد عن 100%

    profiles_list.append({
        'user_id': user_id,
        'dest_cash_out_ratio': ratio,
        'dest_unique_senders': senders
    })

final_profiles = pd.DataFrame(profiles_list)
print("تم إنشاء البروفايلات السلوكية بنجاح.")

# --- الخطوة 3: دمج الميزات (Features) مع المعاملات ---
print("\n--- الخطوة 3: دمج الميزات مع المعاملات ---")
df_model_data = pd.merge(df_smart_sample, final_profiles, left_on='nameDest', right_on='user_id', how='left')
df_model_data = pd.merge(df_model_data, final_profiles, left_on='nameOrig', right_on='user_id', how='left', suffixes=('_dest', '_orig'))

df_model_data['dest_cash_out_ratio_dest'] = df_model_data['dest_cash_out_ratio_dest'].fillna(0)
df_model_data['dest_unique_senders_dest'] = df_model_data['dest_unique_senders_dest'].fillna(0)
df_model_data['dest_cash_out_ratio_orig'] = df_model_data['dest_cash_out_ratio_orig'].fillna(0)
df_model_data['dest_unique_senders_orig'] = df_model_data['dest_unique_senders_orig'].fillna(0)

# --- الخطوة 4: تدريب نموذج (Unsupervised) ---
print("\n--- الخطوة 4: تدريب نموذج Isolation Forest ---")

features = [
    'amount',
    'dest_cash_out_ratio_dest', # نسبة سحب المستلم
    'dest_unique_senders_dest', # عدد مرسلي المستلم
    'dest_cash_out_ratio_orig', # نسبة سحب المرسل
    'dest_unique_senders_orig'  # عدد مرسلي المرسل
]
df_model_data['type_encoded'] = df_model_data['type'].astype('category').cat.codes
features.append('type_encoded')

X = df_model_data[features]
y_true = df_model_data['isFraud'] # "الإجابة الصحيحة"

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

contamination = y_true.mean()
print(f"نسبة الاحتيال (Contamination) في العينة الذكية: {contamination:.2%}")

model = IsolationForest(contamination=contamination, random_state=42)
model.fit(X_scaled)
predictions = model.predict(X_scaled) # -1 = شاذ, 1 = طبيعي
print("تم تدريب النموذج.")

# --- الخطوة 5: الإجابة على سؤالك ---
print("\n--- الخطوة 5: تحديد الحسابات الشاذة (Anomalous Accounts) ---")

# إضافة "تخمين" النموذج إلى البيانات
# 1 = شاذ, 0 = طبيعي
df_model_data['anomaly_prediction'] = [1 if p == -1 else 0 for p in predictions]

# إيجاد "المعاملات" التي حكم عليها النموذج بأنها شاذة
anomalous_transactions = df_model_data[df_model_data['anomaly_prediction'] == 1]

# إيجاد "أسماء الحسابات" (المرسلين والمستلمين) المتورطة في هذه المعاملات الشاذة
anomalous_dest_accounts = anomalous_transactions['nameDest'].unique()
anomalous_orig_accounts = anomalous_transactions['nameOrig'].unique()

all_anomalous_accounts = np.union1d(anomalous_dest_accounts, anomalous_orig_accounts)

print(f"\n[النتيجة النهائية]: وجد النموذج {len(all_anomalous_accounts)} حساباً 'شاذاً' (Anomalous).")

# طباعة عينة من 20 حساباً اعتبرها النموذج "شاذة"
print("عينة من الحسابات التي اعتبرها النموذج 'شاذة':")
print(all_anomalous_accounts[:20])

# --- للمقارنة: ما هي الحسابات الاحتيالية "الحقيقية"؟ ---
print("\n--- للمقارنة (الحقيقة) ---")
print(f"عدد الحسابات الاحتيالية 'الحقيقية' (isFraud=1) كان: {len(all_fraud_user_ids)}")
print("عينة من الحسابات الاحتيالية 'الحقيقية':")
print(all_fraud_user_ids[:20])

# حساب F1-Score للتحقق من الجودة
f1 = f1_score(y_true, df_model_data['anomaly_prediction'])
print(f"\nدرجة F1-Score للنموذج (للتأكيد): {f1:.2%}")

print("\n--- اكتمل الكود المستقر ---")

--- بدء التنفيذ (الكود المستقر) ---
الهدف: تحديد الحسابات الشاذة (Anomalous Accounts) بناءً على سلوكها.
تم تحميل الملف الكامل بنجاح (1048575 صف).

--- الخطوة 1: إنشاء 'العينة الذكية' ---
تم إنشاء 'العينة الذكية' النهائية بحجم 504964 صف.

--- الخطوة 2: بناء البروفايلات السلوكية ---
تم إنشاء البروفايلات السلوكية بنجاح.

--- الخطوة 3: دمج الميزات مع المعاملات ---

--- الخطوة 4: تدريب نموذج Isolation Forest ---
نسبة الاحتيال (Contamination) في العينة الذكية: 0.23%
تم تدريب النموذج.

--- الخطوة 5: تحديد الحسابات الشاذة (Anomalous Accounts) ---

[النتيجة النهائية]: وجد النموذج 1950 حساباً 'شاذاً' (Anomalous).
عينة من الحسابات التي اعتبرها النموذج 'شاذة':
['C1000820773' 'C1000839468' 'C1001444586' 'C10015111' 'C1003526443'
 'C1003775387' 'C1007717381' 'C1008947638' 'C1010765614' 'C1011097249'
 'C1013511446' 'C1013700132' 'C1014154376' 'C1015888357' 'C1016521533'
 'C1017653240' 'C1018394275' 'C1021713645' 'C102174220' 'C1022269511']

--- للمقارنة (الحقيقة) ---
عدد الحسابات الاحتيالية 'الحقيقية

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
# --- (التصليح هنا: تم استيراد النموذج الصحيح) ---
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import warnings

# تجاهل التحذيرات غير الهامة
warnings.filterwarnings('ignore', category=UserWarning)

print("--- بدء التنفيذ (الكود الأذكى: مع ميزة 'الوقت') ---")
print("الهدف: تحسين F1-Score بإضافة 'متوسط وقت السحب' (avg_time_to_cash_out).")

# --- تحميل البيانات ---
# تأكدي أن هذا الاسم مطابق لاسم الملف الذي رفعتِه
file_path = 'PS_20174392719_1491204439457_log24.csv'
try:
    df = pd.read_csv(file_path)
    print(f"تم تحميل الملف الكامل بنجاح ({len(df)} صف).")
except Exception as e:
    print(f"حدث خطأ أثناء تحميل البيانات: {e}")
    exit()

# --- الخطوة 1: إنشاء "العينة الذكية" (Smart Sample) ---
print("\n--- الخطوة 1: إنشاء 'العينة الذكية' ---")
df_fraud = df[df['isFraud'] == 1]
fraud_dest_ids = df_fraud['nameDest'].unique()
fraud_orig_ids = df_fraud['nameOrig'].unique()
all_fraud_user_ids = np.union1d(fraud_dest_ids, fraud_orig_ids)
df_fraud_lifecycle = df[
    df['nameOrig'].isin(all_fraud_user_ids) |
    df['nameDest'].isin(all_fraud_user_ids)
]
df_normal = df[df['isFraud'] == 0]
sample_size = min(500000, len(df_normal))
df_normal_sample = df_normal.sample(n=sample_size, random_state=42)
df_smart_sample = pd.concat([df_fraud_lifecycle, df_normal_sample]).drop_duplicates(keep='first')
print(f"تم إنشاء 'العينة الذكية' النهائية بحجم {len(df_smart_sample)} صف.")

# --- الخطوة 2: بناء البروفايلات السلوكية (بما في ذلك 'الوقت') ---
print("\n--- الخطوة 2: بناء البروفايلات السلوكية (الكاملة) ---")

# (أ) حساب الإحصائيات الأساسية (النسبة، عدد المرسلين)
print("   (أ) حساب نسبة السحب وعدد المرسلين...")
df_received = df_smart_sample[df_smart_sample['type'].isin(['TRANSFER', 'CASH_IN'])]
total_received = df_received.groupby('nameDest')['amount'].sum().to_dict()
unique_senders = df_received.groupby('nameDest')['nameOrig'].nunique().to_dict()
df_cashed_out = df_smart_sample[df_smart_sample['type'] == 'CASH_OUT']
total_cashed_out = df_cashed_out.groupby('nameOrig')['amount'].sum().to_dict()

all_user_ids = set(total_received.keys()) | set(total_cashed_out.keys()) | set(unique_senders.keys())
profiles_list = []
for user_id in all_user_ids:
    received = total_received.get(user_id, 0)
    cashed_out = total_cashed_out.get(user_id, 0)
    senders = unique_senders.get(user_id, 0)
    ratio = (cashed_out / (received + 1e-6))
    ratio = min(ratio, 1.0)
    profiles_list.append({
        'user_id': user_id,
        'dest_cash_out_ratio': ratio,
        'dest_unique_senders': senders
    })
final_profiles = pd.DataFrame(profiles_list)

# (ب) حساب متوسط وقت السحب (الميزة الأذكى باستخدام 'step')
print("   (ب) حساب متوسط وقت السحب (avg_time_to_cash_out)...")
df_transfers = df_smart_sample[df_smart_sample['type'] == 'TRANSFER'][['step', 'nameDest']]
df_cashouts = df_smart_sample[df_smart_sample['type'] == 'CASH_OUT'][['step', 'nameOrig']]
df_transfers.rename(columns={'nameDest': 'user_id'}, inplace=True)
df_cashouts.rename(columns={'nameOrig': 'user_id'}, inplace=True)
df_transfers['tx_type'] = 'TRANSFER_IN'
df_cashouts['tx_type'] = 'CASH_OUT'

user_log = pd.concat([df_transfers, df_cashouts]).sort_values(by=['user_id', 'step'])
user_log['prev_step'] = user_log.groupby('user_id')['step'].shift(1)
user_log['prev_type'] = user_log.groupby('user_id')['tx_type'].shift(1)

# (تصحيح الخطأ السابق)
user_log['time_since_transfer'] = user_log['step'] - user_log['prev_step']
is_pattern = (user_log['tx_type'] == 'CASH_OUT') & (user_log['prev_type'] == 'TRANSFER_IN')
pattern_times = user_log[is_pattern]

avg_time_profile = pattern_times.groupby('user_id')['time_since_transfer'].mean().reset_index()
avg_time_profile.columns = ['user_id', 'avg_time_to_cash_out']

# (ج) تجميع البروفايلات النهائية
print("   (ج) تجميع البروفايلات النهائية...")
final_profiles = pd.merge(final_profiles, avg_time_profile, on='user_id', how='left')
# ملء القيم الفارغة: إذا لم يتبع المستخدم النمط (NaN)، نضع وقتاً طويلاً (مثل 999)
final_profiles['avg_time_to_cash_out'] = final_profiles['avg_time_to_cash_out'].fillna(999)
print("تم إنشاء البروفايلات السلوكية المعقدة بنجاح.")

# --- الخطوة 3: دمج الميزات (Features) مع المعاملات ---
print("\n--- الخطوة 3: دمج الميزات مع المعاملات ---")
df_model_data = pd.merge(df_smart_sample, final_profiles, left_on='nameDest', right_on='user_id', how='left')
df_model_data = pd.merge(df_model_data, final_profiles, left_on='nameOrig', right_on='user_id', how='left', suffixes=('_dest', '_orig'))

# ملء القيم الفارغة (NaNs)
df_model_data['dest_cash_out_ratio_dest'] = df_model_data['dest_cash_out_ratio_dest'].fillna(0)
df_model_data['dest_unique_senders_dest'] = df_model_data['dest_unique_senders_dest'].fillna(0)
df_model_data['avg_time_to_cash_out_dest'] = df_model_data['avg_time_to_cash_out_dest'].fillna(999)
df_model_data['dest_cash_out_ratio_orig'] = df_model_data['dest_cash_out_ratio_orig'].fillna(0)
df_model_data['dest_unique_senders_orig'] = df_model_data['dest_unique_senders_orig'].fillna(0)
df_model_data['avg_time_to_cash_out_orig'] = df_model_data['avg_time_to_cash_out_orig'].fillna(999)

# --- الخطوة 4: تدريب نموذج (Unsupervised) ---
print("\n--- الخطوة 4: تدريب نموذج Isolation Forest (بالميزات الجديدة) ---")

features = [
    'amount',
    'dest_cash_out_ratio_dest',
    'dest_unique_senders_dest',
    'avg_time_to_cash_out_dest', # <-- الميزة الجديدة (للمستلم)
    'dest_cash_out_ratio_orig',
    'dest_unique_senders_orig',
    'avg_time_to_cash_out_orig'  # <-- الميزة الجديدة (للمرسل)
]
df_model_data['type_encoded'] = df_model_data['type'].astype('category').cat.codes
features.append('type_encoded')

X = df_model_data[features]
y_true = df_model_data['isFraud'] # "الإجابة الصحيحة"

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

contamination = y_true.mean()
print(f"نسبة الاحتيال (Contamination) في العينة الذكية: {contamination:.2%}")

# --- (هذا هو السطر الذي تم تصحيحه) ---
model = IsolationForest(contamination=contamination, random_state=42)
model.fit(X_scaled)
predictions = model.predict(X_scaled) # -1 = شاذ, 1 = طبيعي
print("تم تدريب النموذج.")

# --- الخطوة 5: التقييم (المقارنة) ---
print("\n--- الخطوة 5: تقييم النموذج (الدرجة الجديدة) ---")

y_pred = [1 if p == -1 else 0 for p in predictions]
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print(f"Precision (الدقة): {precision:.2%}")
print(f"Recall (قوة الالتقاط): {recall:.2%}")
print(f"F1-Score (الدرجة النهائية الجديدة): {f1:.2%}")
print("\nConfusion Matrix (مصفوفة الأخطاء):")
print(cm)

print("\n--- اكتمل الكود الأذكى ---")

--- بدء التنفيذ (الكود الأذكى: مع ميزة 'الوقت') ---
الهدف: تحسين F1-Score بإضافة 'متوسط وقت السحب' (avg_time_to_cash_out).
تم تحميل الملف الكامل بنجاح (1048575 صف).

--- الخطوة 1: إنشاء 'العينة الذكية' ---
تم إنشاء 'العينة الذكية' النهائية بحجم 504964 صف.

--- الخطوة 2: بناء البروفايلات السلوكية (الكاملة) ---
   (أ) حساب نسبة السحب وعدد المرسلين...
   (ب) حساب متوسط وقت السحب (avg_time_to_cash_out)...
   (ج) تجميع البروفايلات النهائية...
تم إنشاء البروفايلات السلوكية المعقدة بنجاح.

--- الخطوة 3: دمج الميزات مع المعاملات ---

--- الخطوة 4: تدريب نموذج Isolation Forest (بالميزات الجديدة) ---
نسبة الاحتيال (Contamination) في العينة الذكية: 0.23%
تم تدريب النموذج.

--- الخطوة 5: تقييم النموذج (الدرجة الجديدة) ---
Precision (الدقة): 3.13%
Recall (قوة الالتقاط): 3.06%
F1-Score (الدرجة النهائية الجديدة): 3.10%

Confusion Matrix (مصفوفة الأخطاء):
[[502740   1082]
 [  1107     35]]

--- اكتمل الكود الأذكى ---


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
# --- (التغيير: سنستخدم نموذج Supervised) ---
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
# --- (التغيير: نحتاج لتقسيم البيانات) ---
from sklearn.model_selection import train_test_split
import warnings

# تجاهل التحذيرات غير الهامة
warnings.filterwarnings('ignore', category=UserWarning)

print("--- بدء التنفيذ (النموذج الناجح: Supervised) ---")
print("الهدف: إثبات أن 'الميزات السلوكية' تنجح مع نموذج Supervised.")

# --- تحميل البيانات ---
file_path = 'PS_20174392719_1491204439457_log24.csv'
try:
    df = pd.read_csv(file_path)
    print(f"تم تحميل الملف الكامل بنجاح ({len(df)} صف).")
except Exception as e:
    print(f"حدث خطأ أثناء تحميل البيانات: {e}")
    exit()

# --- الخطوة 1: إنشاء "العينة الذكية" (Smart Sample) ---
# (نفس الكود لبناء عينة متوازنة للتدريب)
print("\n--- الخطوة 1: إنشاء 'العينة الذكية' ---")
df_fraud = df[df['isFraud'] == 1]
fraud_dest_ids = df_fraud['nameDest'].unique()
fraud_orig_ids = df_fraud['nameOrig'].unique()
all_fraud_user_ids = np.union1d(fraud_dest_ids, fraud_orig_ids)
df_fraud_lifecycle = df[
    df['nameOrig'].isin(all_fraud_user_ids) |
    df['nameDest'].isin(all_fraud_user_ids)
]
df_normal = df[df['isFraud'] == 0]
sample_size = min(500000, len(df_normal))
df_normal_sample = df_normal.sample(n=sample_size, random_state=42)
df_smart_sample = pd.concat([df_fraud_lifecycle, df_normal_sample]).drop_duplicates(keep='first')
print(f"تم إنشاء 'العينة الذكية' النهائية بحجم {len(df_smart_sample)} صف.")

# --- الخطوة 2: بناء البروفايلات السلوكية (الميزات القوية) ---
print("\n--- الخطوة 2: بناء البروفايلات السلوكية ---")
df_received = df_smart_sample[df_smart_sample['type'].isin(['TRANSFER', 'CASH_IN'])]
total_received = df_received.groupby('nameDest')['amount'].sum().to_dict()
unique_senders = df_received.groupby('nameDest')['nameOrig'].nunique().to_dict()
df_cashed_out = df_smart_sample[df_smart_sample['type'] == 'CASH_OUT']
total_cashed_out = df_cashed_out.groupby('nameOrig')['amount'].sum().to_dict()

all_user_ids = set(total_received.keys()) | set(total_cashed_out.keys()) | set(unique_senders.keys())
profiles_list = []
for user_id in all_user_ids:
    received = total_received.get(user_id, 0)
    cashed_out = total_cashed_out.get(user_id, 0)
    senders = unique_senders.get(user_id, 0)
    ratio = (cashed_out / (received + 1e-6))
    ratio = min(ratio, 1.0)
    profiles_list.append({
        'user_id': user_id,
        'dest_cash_out_ratio': ratio,
        'dest_unique_senders': senders
    })
final_profiles = pd.DataFrame(profiles_list)
print("تم إنشاء البروفايلات السلوكية بنجاح.")

# --- الخطوة 3: دمج الميزات (Features) مع المعاملات ---
print("\n--- الخطوة 3: دمج الميزات مع المعاملات ---")
df_model_data = pd.merge(df_smart_sample, final_profiles, left_on='nameDest', right_on='user_id', how='left')
df_model_data = pd.merge(df_model_data, final_profiles, left_on='nameOrig', right_on='user_id', how='left', suffixes=('_dest', '_orig'))

df_model_data['dest_cash_out_ratio_dest'] = df_model_data['dest_cash_out_ratio_dest'].fillna(0)
df_model_data['dest_unique_senders_dest'] = df_model_data['dest_unique_senders_dest'].fillna(0)
df_model_data['dest_cash_out_ratio_orig'] = df_model_data['dest_cash_out_ratio_orig'].fillna(0)
df_model_data['dest_unique_senders_orig'] = df_model_data['dest_unique_senders_orig'].fillna(0)

# --- الخطوة 4: تجهيز بيانات التدريب والاختبار (Supervised) ---
print("\n--- الخطوة 4: تجهيز بيانات التدريب والاختبار ---")

features = [
    'amount',
    'dest_cash_out_ratio_dest', # نسبة سحب المستلم
    'dest_unique_senders_dest', # عدد مرسلي المستلم
    'dest_cash_out_ratio_orig', # نسبة سحب المرسل
    'dest_unique_senders_orig'  # عدد مرسلي المرسل
]
df_model_data['type_encoded'] = df_model_data['type'].astype('category').cat.codes
features.append('type_encoded')

X = df_model_data[features]
y_true = df_model_data['isFraud'] # "الإجابة الصحيحة"

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# (التغيير: نقسم البيانات 70% للتدريب و 30% للاختبار)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_true, test_size=0.3, random_state=42, stratify=y_true)
print(f"تم تقسيم البيانات إلى {len(X_train)} صف للتدريب و {len(X_test)} صف للاختبار.")

# --- الخطوة 5: تدريب نموذج (Supervised RandomForest) ---
print("\n--- الخطوة 5: تدريب نموذج RandomForest (Supervised) ---")

# (class_weight='balanced' مهم جداً للبيانات غير المتوازنة)
model = RandomForestClassifier(random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

# الاختبار على بيانات "جديدة" (X_test) لم يرها النموذج من قبل
predictions = model.predict(X_test)
print("تم تدريب النموذج.")

# --- الخطوة 6: التقييم (النتيجة الناجحة) ---
print("\n--- الخطوة 6: تقييم النموذج (النتيجة الجديدة) ---")

f1 = f1_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
cm = confusion_matrix(y_test, predictions)

print("!!! النتائج على 'بيانات الاختبار' (بيانات جديدة لم يرها النموذج) !!!")
print(f"Precision (الدقة): {precision:.2%}")
print(f"Recall (قوة الالتقاط): {recall:.2%}")
print(f"F1-Score (الدرجة النهائية الجديدة): {f1:.2%}")
print("\nConfusion Matrix (مصفوفة الأخطاء):")
print(cm)

# --- (الخطوة الأهم: لماذا نجح النموذج؟) ---
print("\n--- أهمية الميزات (لماذا نجح النموذج؟) ---")
feature_imp = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
print(feature_imp)

print("\n--- اكتمل الكود الناجح ---")

--- بدء التنفيذ (النموذج الناجح: Supervised) ---
الهدف: إثبات أن 'الميزات السلوكية' تنجح مع نموذج Supervised.
تم تحميل الملف الكامل بنجاح (1048575 صف).

--- الخطوة 1: إنشاء 'العينة الذكية' ---
تم إنشاء 'العينة الذكية' النهائية بحجم 504964 صف.

--- الخطوة 2: بناء البروفايلات السلوكية ---
تم إنشاء البروفايلات السلوكية بنجاح.

--- الخطوة 3: دمج الميزات مع المعاملات ---

--- الخطوة 4: تجهيز بيانات التدريب والاختبار ---
تم تقسيم البيانات إلى 353474 صف للتدريب و 151490 صف للاختبار.

--- الخطوة 5: تدريب نموذج RandomForest (Supervised) ---
تم تدريب النموذج.

--- الخطوة 6: تقييم النموذج (النتيجة الجديدة) ---
!!! النتائج على 'بيانات الاختبار' (بيانات جديدة لم يرها النموذج) !!!
Precision (الدقة): 27.03%
Recall (قوة الالتقاط): 20.41%
F1-Score (الدرجة النهائية الجديدة): 23.26%

Confusion Matrix (مصفوفة الأخطاء):
[[150958    189]
 [   273     70]]

--- أهمية الميزات (لماذا نجح النموذج؟) ---
amount                      0.539584
type_encoded                0.234897
dest_unique_senders_dest    0.143611